In [1]:

import numpy as np
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import pandas as pd
import math
from numpy import diff
from numpy import gradient
import numpy as np
from scipy import pi
from scipy import fft
from scipy.fft import fft, fftfreq
from scipy import signal
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go
from scipy.signal import find_peaks

from plotly.subplots import make_subplots


In [2]:
df1=pd.read_csv('mtb_phd/minerals/OPPOFindX5Pro-03-15-02/OPPOFindX5Pro-03-15-02.miq', 
                skiprows=57, skipfooter=35,names=['sample', 'Lat','Lon', 'Alt','Time', 'Speed'])

#postion,sensors  = get_data(mtb_phd/minerals/OPPOFindX5Pro-03-15-02/OPPOFindX5Pro-03-15-02)
df=pd.read_csv('mtb_phd/drive-download-20230823T084905Z-001/BrakeAce 20230315180854 Minerals +Torque.csv') #,usecols=[1,2,3,4,5], 
               #names=( 'fork','rearWheel','frontForce','shock','rearForce'), header=0)
               
df

/tmp/ipykernel_12749/1410175824.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df1=pd.read_csv('mtb_phd/minerals/OPPOFindX5Pro-03-15-02/OPPOFindX5Pro-03-15-02.miq',


,Time (s),Distance (m),Latitude (deg),Longitude (deg),Elevation (m),Speed (km/h),Front Brake Power (W),Rear Brake Power (W),Front Brake Torque (Nm),Rear Brake Torque (Nm)
0,38.2,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
1,38.3,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
2,38.4,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
3,38.5,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
4,38.6,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2302,268.4,1106,-38.16651,176.30820,460,3.9,0.0,0.0,0.0,0.0
2303,268.5,1107,-38.16651,176.30820,460,3.9,0.0,0.0,0.0,0.0
2304,268.6,1107,-38.16651,176.30820,460,3.9,0.0,0.0,0.0,0.0
2305,268.7,1107,-38.16651,176.30820,460,3.9,0.0,0.0,0.0,0.0


In [3]:
df.set_index('Time (s)', inplace=True, drop=False)
df

,Time (s),Distance (m),Latitude (deg),Longitude (deg),Elevation (m),Speed (km/h),Front Brake Power (W),Rear Brake Power (W),Front Brake Torque (Nm),Rear Brake Torque (Nm)
Time (s),,,,,,,,,,
38.2,38.2,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
38.3,38.3,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
38.4,38.4,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
38.5,38.5,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
38.6,38.6,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
268.4,268.4,1106,-38.16651,176.30820,460,3.9,0.0,0.0,0.0,0.0
268.5,268.5,1107,-38.16651,176.30820,460,3.9,0.0,0.0,0.0,0.0
268.6,268.6,1107,-38.16651,176.30820,460,3.9,0.0,0.0,0.0,0.0


In [4]:
df.index=pd.to_timedelta(df.index, unit='s') #format='%S.%3f'
df

,Time (s),Distance (m),Latitude (deg),Longitude (deg),Elevation (m),Speed (km/h),Front Brake Power (W),Rear Brake Power (W),Front Brake Torque (Nm),Rear Brake Torque (Nm)
Time (s),,,,,,,,,,
0 days 00:00:38.200000,38.2,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
0 days 00:00:38.300000,38.3,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
0 days 00:00:38.400000,38.4,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
0 days 00:00:38.500000,38.5,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
0 days 00:00:38.600000,38.6,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
0 days 00:04:28.400000,268.4,1106,-38.16651,176.30820,460,3.9,0.0,0.0,0.0,0.0
0 days 00:04:28.500000,268.5,1107,-38.16651,176.30820,460,3.9,0.0,0.0,0.0,0.0
0 days 00:04:28.600000,268.6,1107,-38.16651,176.30820,460,3.9,0.0,0.0,0.0,0.0


In [5]:
df=df.resample('0.005S').bfill().dropna()#. interpolate(method='polynomial', order=2)
#df= df.reset_index(drop=True)
#df['Time (s)'].interpolate(method='polynomial', order=2)

In [136]:
#df.interpolate(method='polynomial', order=2)

In [6]:
df

,Time (s),Distance (m),Latitude (deg),Longitude (deg),Elevation (m),Speed (km/h),Front Brake Power (W),Rear Brake Power (W),Front Brake Torque (Nm),Rear Brake Torque (Nm)
Time (s),,,,,,,,,,
0 days 00:00:38.200000,38.2,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
0 days 00:00:38.205000,38.3,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
0 days 00:00:38.210000,38.3,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
0 days 00:00:38.215000,38.3,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
0 days 00:00:38.220000,38.3,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
0 days 00:04:28.780000,268.8,1108,-38.16651,176.30820,460,3.9,0.0,0.0,0.0,0.0
0 days 00:04:28.785000,268.8,1108,-38.16651,176.30820,460,3.9,0.0,0.0,0.0,0.0
0 days 00:04:28.790000,268.8,1108,-38.16651,176.30820,460,3.9,0.0,0.0,0.0,0.0


In [7]:
    # size=len(df1.index)
    # timedata,interval=np.linspace(0,size/200,num=size,retstep=True, endpoint=False) #create timestamps
    # df1['time']=np.round(timedata,3) # add to dataframe, round to 0.001s
df1['elapsed_time'] = df1['Time'] - df1['Time'][0]

In [8]:
df1.set_index(df1['elapsed_time'], inplace=True, drop=False)

In [9]:
print(df1)

              sample        Lat         Lon         Alt          Time  \
elapsed_time                                                            
0.000              3 -38.170776  176.302207  634.598145  1.678857e+09   
1.195              4 -38.170746  176.302225  634.472534  1.678857e+09   
1.792              5 -38.170705  176.302245  634.472534  1.678857e+09   
2.298              6 -38.170700  176.302249  633.794678  1.678857e+09   
2.867              7 -38.170688  176.302255  627.541016  1.678857e+09   
...              ...        ...         ...         ...           ...   
233.645          296 -38.166518  176.308504  454.841675  1.678857e+09   
234.242          297 -38.166518  176.308506  455.514648  1.678857e+09   
234.839          298 -38.166518  176.308508  455.976929  1.678857e+09   
235.342          299 -38.166517  176.308509  455.976929  1.678857e+09   
235.848          300 -38.166517  176.308510  455.976929  1.678857e+09   

                 Speed  elapsed_time  
elapsed_tim

In [10]:
df1.index=pd.to_timedelta(df1.index, unit='s')
# df1=df1.resample('0.005S')#.bfill()

In [11]:
df1

,sample,Lat,Lon,Alt,Time,Speed,elapsed_time
elapsed_time,,,,,,,
0 days 00:00:00,3,-38.170776,176.302207,634.598145,1.678857e+09,2.345496,0.000
0 days 00:00:01.195000172,4,-38.170746,176.302225,634.472534,1.678857e+09,2.939015,1.195
0 days 00:00:01.792000055,5,-38.170705,176.302245,634.472534,1.678857e+09,2.838630,1.792
0 days 00:00:02.298000097,6,-38.170700,176.302249,633.794678,1.678857e+09,2.679965,2.298
0 days 00:00:02.867000103,7,-38.170688,176.302255,627.541016,1.678857e+09,2.634642,2.867
...,...,...,...,...,...,...,...
0 days 00:03:53.645000219,296,-38.166518,176.308504,454.841675,1.678857e+09,0.193711,233.645
0 days 00:03:54.242000103,297,-38.166518,176.308506,455.514648,1.678857e+09,0.135244,234.242
0 days 00:03:54.839000225,298,-38.166518,176.308508,455.976929,1.678857e+09,0.086141,234.839


In [78]:
df2=df1.resample('0.005S').bfill() #.dropna()# .interpolate(method='slinear', order=2)

In [79]:
df2=df2.dropna()


In [72]:
# peaks= df[' Speed (km/h)'].idxmax()
# # troughs, _= find_peaks(-df['force'],distance=8000)
# peaks1=df2['Speed'].idxmax()
# print(peaks,peaks1)
# t=peaks-peaks1
# t=t/np.timedelta64(1, "s")
# print(t)

0 days 00:02:10.905000 0 days 00:01:23.275000


In [80]:
peaks,_= find_peaks(df[' Speed (km/h)'], distance=500)
# troughs, _= find_peaks(-df['force'],distance=8000)
peaks1,_= find_peaks(df2['Speed'], distance=500)
print(peaks,peaks1)

[ 1970  3270  4340  5960  6970  7620  9340 11150 12140 13110 14090 14750
 15740 16740 17760 18570 21150 21750 22340 23450 24350 25300 26550 27440
 28500 29300 30310 31040 31550 32140 32960 34100 34940 36700 37350 39720
 41090 42150 43150 43730 44350 45900] [  120  1512  2154  2954  3583  4154  5187  5754  7554  9354 10354 11354
 12285 13009 13954 14954 15954 16754 19354 19954 20554 21709 22456 23569
 24233 25227 25954 26742 27546 28558 29230 29754 30354 31154 32354 33154
 34954 35554 37954 39335 40354 41368 41954 42591 44164]


In [81]:
c=df['Time (s)'].iloc[peaks[0]]
d=df2['elapsed_time'].iloc[peaks1[0]]
print(c,d)

48.1 1.195000171661377


In [82]:
df2['elapsed_time']=df2['elapsed_time']+(c-d-0.177)

In [76]:
# df2['elapsed_time']=df2['elapsed_time']+t

In [83]:
fig=px.line(df2,x='elapsed_time', y='Speed')
fig.add_trace(go.Scatter(x=df['Time (s)'], y=df[' Speed (km/h)']))

fig.show()

In [18]:
# # distance, path = fastdtw(b.T, a.T, dist=euclidean) #radius=100, 
# # print(distance)
# print(path)
# len(path)

344341.31365444406


In [37]:
# result = []
# for i in range(0,len(path)):
#     result.append([df['Time (s)'].iloc[path[i][0]],
#     df1['Speed'].iloc[path[i][1]],
#     df[' Speed (km/h)'].iloc[path[i][0]]])
#     # b[1][path[i][0]],
#     # a[1][path[i][0]]])
# df_sync = pd.DataFrame(data=result,columns=['Time','suspension','brake']).dropna()#
# df_sync = df_sync.drop_duplicates(subset=['Time'])
# df_sync = df_sync.sort_values(by='Time')
# df_sync = df_sync.reset_index(drop=True)
#df_sync.to_csv('C:/Users/.../synchronized_dataset.csv',index=False)
# shifted=[]
# shiftedb=[]
# for m in path:
#     print(m)
#     d=m[0]
#     print(d)
#     e=a[d]
#     shifted.append(e)

# fig=px.line(df_sync, x='Time', y='suspension')
# fig.add_trace(go.Scatter(x=df_sync['Time'], y=df_sync['brake']))

# fig.show()
